## 轉移學習練習
回顧 CNN 圖形辨識模型

### 1. 初始準備

In [1]:
%env KERAS_BACKEND = tensorflow
%matplotlib inline

import numpy as np
import matplotlib.pyplot as np

# keras functions
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

# keras dataset
from keras.datasets import mnist

# keras utilis function
from keras.utils import np_utils

env: KERAS_BACKEND=tensorflow


### 2. 讀入 MNIST 數據庫
MNIST 是有一堆 0~9 的手寫數字圖庫，有 6 萬筆訓練資料，1 萬筆測試資料。
MNIST 是 Deep Learning 最有名的範例。

#### 2.1 由 Keras 讀入 MNIST
Keras 提供 MNIST，可直接讀入

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

看資料的長相

In [3]:
print('there are %d training data with size %d x %d' %x_train.shape)
print('there are %d testing data with size %d x %d' %x_test.shape)

there are 60000 training data with size 28 x 28
there are 10000 testing data with size 28 x 28


### 2.2 輸入格式整理
現在要用 CNN 學手寫辨識，因為 CNN 模型的資料需要多一個 channel (通道數)，因此要用 `reshape` 調教

In [4]:
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

為了後面需要，先將數字 0 和 1 的資料分別抓出來

In [5]:
x_train_01 = x_train[y_train <= 1]
x_test_01 = x_test[y_test <= 1]

並將 label 轉換成 one-hot encoding 的形式

In [6]:
y_train_10 = np_utils.to_categorical(y_train, 10)
y_test_10 = np_utils.to_categorical(y_test, 10)

y_train_01 = y_train[y_train <= 1]
y_train_01 = np_utils.to_categorical(y_train_01, 2)

y_test_01 = y_test[y_test <= 1]
y_test_01 = np_utils.to_categorical(y_test_01, 2)

養成好習慣，適時確認資料的大小以確保資料的一致性

In [7]:
x_train_01.shape, x_test_01.shape

((12665, 28, 28, 1), (2115, 28, 28, 1))

In [8]:
y_train_01.shape, y_test_01.shape

((12665, 2), (2115, 2))

## 3. 回顧 CNN 圖形辨識模型

經典的 CNN 圖形辨識模型 LeNet-5 是一個由兩層卷基層加三層全連接層所建立的神經網路，在第二單元所建立的 CNN 模型如下：

* 起始為 3 個 convolutional block
    * 每個 convolutional block 為 1 個 2D Convolution + ReLU + 1 個 2D MacPooling
    * 2D Convolution 的數量為 32, 64, 128
    * 每個 2D Convolution 的 `kernal_size` 為 3 或 (3,3)，`padding` 使用 `same`
    * 每個 2D MaxPooling 的 `pool_size` 為 2 或 (2,2)，`padding` 使用 `same`
* 將輸出結果 `Flatten` 後，接著兩層全連接層，神經元個數分別為 200 和 10 (數字的類別總數)

當時建立的是一個具有三層卷基層加兩層全連接的神經網路，可以看成是 LeNet-5 的一種變形
本單元的內容，可以使用下列方式使用 Sequential 重新建構第二單元的 CNN 模型

In [9]:
# put 3 conv. blocks together, called conv_layer.
conv_layer = [Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
             Activation('relu'),
             MaxPooling2D(pool_size=(2, 2)),
             
             Conv2D(64, (3, 3), padding='same'),
             Activation('relu'),
             MaxPooling2D(pool_size=(2, 2)),
             
             Conv2D(128, (3, 3), padding='same'),
             Activation('relu'),
             MaxPooling2D(pool_size=(2, 2))]

# put Flatten, and 2 full-connectd layers together, called fc_layer.
fc_layer = [Flatten(),
           Dense(200),
           Activation('relu'),
           Dense(10),
           Activation('softmax')]

model = Sequential(conv_layer + fc_layer)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
______________________________________

## 4. 保留前三層 convolutional layer 並進行轉移學習

將 MNIST 資料及僅有 0, 1 的部分取出來，希望透過轉移學習建立一個類似 LeNet-5 的 0, 1 圖形辨識模型
將下列三個 **None** 的部分進行修改，以轉移學習建立新的模型

In [25]:
new_fc_layer = [Flatten(),
               # Design your own fully connected structures
               Dense(100),
               Activation('relu'),
               Dense(2), # Hint: how many classes in new dataset?
               # Remember put currect number of unit for output
               Activation('softmax')]

model_0_to_1 = Sequential(conv_layer + new_fc_layer)
model_0_to_1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)        

將下列的 **None** 進行修改，將借來的神經網路**冷凍**起來：

In [35]:
for layer in conv_layer:
    layer.trainable = False

冷凍後神經網路的 summary 會有些變化

In [36]:
model_0_to_1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)        

/Users/john.tung/anaconda3/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


接著訓練有部分架構跟別人借的 0, 1 手寫辨識模型

In [37]:
model_0_to_1.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=['accuracy'])

## 5. 訓練

In [28]:
x_train_01.shape, y_train_01.shape

((12665, 28, 28, 1), (12665, 2))

In [38]:
model_0_to_1.fit(x_train_01, y_train_01, batch_size=100, epochs=5)

Epoch 1/5
12665/12665 [==============================] - 8s 648us/step - loss: 0.5323 - accuracy: 0.4677
Epoch 2/5
12665/12665 [==============================] - 8s 651us/step - loss: 0.5323 - accuracy: 0.4677
Epoch 3/5
12665/12665 [==============================] - 9s 706us/step - loss: 0.5323 - accuracy: 0.4677
Epoch 4/5
12665/12665 [==============================] - 8s 655us/step - loss: 0.5323 - accuracy: 0.4677
Epoch 5/5
12665/12665 [==============================] - 8s 665us/step - loss: 0.5323 - accuracy: 0.4677


In [39]:
score = model_0_to_1.evaluate(x_test_01, y_test_01)

2115/2115 [==============================] - 2s 854us/step


In [40]:
print('測試資料的 loss：', score[0])
print('測試資料的正確率：', score[1])

測試資料的 loss： 0.53664302603291
測試資料的正確率： 0.46335697174072266


## 6. 第二個轉移學習得到的神經網路
如果模型大部分的權重已經訓練好並冷凍起來，轉移學習可以大幅減少訓練時間且訓練會更快收斂